In [1]:
import awswrangler as wr
import pandas as pd

from data_sci_toolkit.aws_tools import permission_tools 

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

/opt/miniconda3/envs/stellar/lib/python3.8/site-packages/requests/__init__.py:109: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (5.0.0)/charset_normalizer (2.0.12) doesn't match a supported version!
  warnings.warn(


In [2]:
SESSION = permission_tools.get_aws_session("Dev-DataScienceAdmin")

Attempting to automatically open the SSO authorization page in your default browser.
If the browser does not open or you wish to use a different device to authorize this request, open the following URL:

https://device.sso.us-east-1.amazonaws.com/

Then enter the code:

GHQC-GQQW
Successfully logged into Start URL: https://stellaralgo.awsapps.com/start#/
Authorized as AROAQBUNWK7XN5DTYWPLJ:rkazmerik@stellaralgo.com


In [3]:
DATABASE = "integrations_milb_aviators"
S3_BUCKET = "s3://dev-athena-testing-us-east-1"

In [4]:
df2 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= """
        WITH customers as (
            select * 
            from integrations_milb_aviators.customers c
            CROSS JOIN UNNEST(account_ids['ticketing'], account_ids['retail']) as t(ticket_ids, retail_ids)
        ),
        ticketing AS (
            select *
            from integrations_milb_aviators.tickets t
            CROSS JOIN UNNEST(resales, forwards, scans) as t(resale_data, forward_data, scan_data)
            where returned_at IS NULL
        )
        
        select * 
        from ticketing t
        INNER JOIN integrations_milb_aviators.ticketing_events e
        on t.event_id = e.id
        INNER JOIN customers c
        on c.ticket_ids = t.account_id limit 10 
    """
)
df2.shape

(10, 76)

In [5]:
df2

,source_system,source_system_group,stlr_reported_revenue,client_property,stlr_product,seat_id,section_name,row_name,seat_num,season_id,event_id,event_name,plan_event_id,ticket_type,comp_code,promo_code,account_id,price_code,price,account_rep_id,account_rep_name,purchased_at,returned_at,updated_at,forwards,resales,scans,resale_data,forward_data,scan_data,source_system.1,source_system_group.1,id,stlr_is_renewal,stlr_game_number,client_property.1,name,full_name,venue,team,start_date,end_date,day_of_week,time,time_of_day,event_type,season_id.1,season,season_year,game_number,min_events,total_events,fse,has_payment_plan,major_category,minor_category,id.1,account_ids,email,first_name,last_name,address,city,state,country,zip,phone,company_name,account_type,account_rep_name.1,demographics,scv_history,scv_updated_at,venues,ticket_ids,retail_ids
0,ticketmaster,ticketing,70.0,milb_aviators,Individual,CF19272BA6C63714EE807CF34C084D4A,DOB105,B,20,63,2977,EV200308,<NA>,Adult,0,<NA>,1365158,E,70.0,16,Erik Eisenberg,2019-12-22 22:27:49.051,NaT,NaT,NaN,NaN,"[{account_id=1365158, scanned_at=null, scanned...",NaN,NaN,"{account_id=1365158, scanned_at=null, scanned_...",ticketmaster,ticketing,2977,False,4,other,EV200308,Cincinnati Vs Chicago 3/8,Las Vegas Ballpark,Cincinnati vs Chicago,2020-03-08,2020-03-08,SUN,13:05:00,Afternoon,Single Event,63,2020 Big League Weekend,2019,4,1,1,0.0,False,Sports,Mlb Pro Baseball,a9ce0d26ab3359a722a46a274e80503a,{ticketing=[1365158]},sabrinalbeckman@gmail.com,Sabrina,Beckman,2816 Turnstone Ridge St,Henderson,Nevada,United States,89044,+1 (702) 408-7729,<NA>,Personal,Erik Eisenberg,NaN,"[13264cf6dc27704ab133b54a828915cc, a12aee207b1...",2023-03-15 18:47:24,"[{name=Las Vegas Ballpark, distance=30.8412158...",1365158,<NA>
1,ticketmaster,ticketing,65.0,milb_aviators,Group,0E1509F0D890E42BAEE616E095C7BD86,DOB106,S,9,63,2976,EV200307,<NA>,Groups $5.00 off,0,<NA>,303479,EG,65.0,28,Bryan Frey,2019-12-17 09:38:34.252,NaT,NaT,"[{source_account_id=303479, account_id=1790134...",NaN,"[{account_id=1790134, scanned_at=null, scanned...",NaN,"{source_account_id=303479, account_id=1790134,...","{account_id=1790134, scanned_at=null, scanned_...",ticketmaster,ticketing,2976,False,3,other,EV200307,Cincinnati Vs Chicago 3/7,Las Vegas Ballpark,Cincinnati vs Chicago,2020-03-07,2020-03-07,SAT,13:05:00,Afternoon,Single Event,63,2020 Big League Weekend,2019,3,1,1,0.0,False,Sports,Mlb Pro Baseball,9a5d2cad45d5d147ca1740eccd2c73a3,"{ticketing=[2281856, 303479], retail=[25683713...",tgaspari@yahoo.com,Tony,Gaspari,2 Tudor City Place Apt # 6ds,New York,New York,United States,10017,+1 (917) 331-8121,<NA>,Personal,Michelle Taggart,NaN,NaN,NaT,"[{name=Las Vegas Ballpark, distance=3603.93475...",303479,<NA>
2,ticketmaster,ticketing,55.0,milb_aviators,Group,5F656E5223C67D7667564ABA1BEDC00E,OFB104,T,18,63,2977,EV200308,<NA>,Groups $5.00 off,0,<NA>,1722159,FG,55.0,28,Bryan Frey,2020-01-14 15:27:35.398,NaT,NaT,"[{source_account_id=1722159, account_id=176386...",NaN,"[{account_id=1763869, scanned_at=null, scanned...",NaN,"{source_account_id=1722159, account_id=1763869...","{account_id=1763869, scanned_at=null, scanned_...",ticketmaster,ticketing,2977,False,4,other,EV200308,Cincinnati Vs Chicago 3/8,Las Vegas Ballpark,Cincinnati vs Chicago,2020-03-08,2020-03-08,SUN,13:05:00,Afternoon,Single Event,63,2020 Big League Weekend,2019,4,1,1,0.0,False,Sports,Mlb Pro Baseball,1c6b99c0f04ec0722a1c9c1b55787db9,"{ticketing=[1722159], retail=[2742625271890]}",gui.ramostx@gmail.com,Guillermo,Ramos,9804 Cantera Ct,Laredo,Texas,United States,78045,+1 (940) 784-3579,<NA>,Season Ticket,Bryan Frey,NaN,NaN,NaT,"[{name=Las Vegas Ballpark, distance=1758.15374...",1722159,2742625271890
3,ticketmaster,ticketing,150.0,milb_aviators,Individual,D91DF468F3299B7539826B955688D529,S12,C,1,63,2977,EV200308,<NA>,Adult,0,<NA>,1724954,P,150.0,16,Erik Eisenberg,2020-08-18 14:42:46.009,NaT,NaT,NaN,NaN,"[{account_id=1724954, scanned_at=null, scanned...",NaN,NaN,"{account_id=1724954, scanned_

In [12]:
sql3 = """with 
customers as (
    select c.id as scv_id, source_system_type, account_id, venue
    from integrations_milb_aviators.customers as c
    cross join unnest(account_ids) as t(source_system_type, source_account_ids)
    cross join unnest(source_account_ids) as a(account_id)
    cross join unnest(venues) as v(venue)
),
ticketing_customers as (
    select 
        c.scv_id, 
        c.venue.distance as distance_to_venue, 
        te.season_year, 
        t.event_id,
        te.start_date as event_date,
        t.stlr_product, 
        t.seat_id,
        t.price,
        t.purchased_at,
        t.scans is not null as was_attended
    from customers as c
    join tickets as t on t.account_id = c.account_id and source_system_type = 'ticketing'
    join ticketing_events as te on te.id = t.event_id
)

select c.*, date_diff('day', first_purchase_date, CURRENT_DATE) as tenure
from (
    select 
        scv_id, 
        distance_to_venue, 
        stlr_product, 
        season_year, 
        max(event_date) as last_event_date,
        count(*) as tickets_purchased, 
        sum(price) as total_spent, 
        (1.0 * sum(case when was_attended then 1 else 0 end)) / count(*) as attendance_percentage,
        max(case when was_attended then event_date else null end) as last_attendance_date,
        (lead(count(*), 1) over (partition by scv_id order by season_year) is not null) as is_next_year_buyer
    from ticketing_customers
    group by scv_id, distance_to_venue, stlr_product, season_year) as c
join (
    select scv_id, min(purchased_at) as first_purchase_date
    from ticketing_customers
    group by scv_id
) as t
on c.scv_id = t.scv_id"""

In [13]:
df3 = wr.athena.read_sql_query(
    boto3_session= SESSION, 
    ctas_approach= False,
    database= DATABASE,
    s3_output= S3_BUCKET,
    sql= sql3
)
df3.shape

(197195, 11)

In [14]:
df3.head()

,scv_id,distance_to_venue,stlr_product,season_year,last_event_date,tickets_purchased,total_spent,attendance_percentage,last_attendance_date,is_next_year_buyer,tenure
0,0006ecf52ebc80dca51e4e62feb5cb57,1.608480,Individual,2019,2019-04-25,2,36.0,1.0,2019-04-25,True,1437
1,0006ecf52ebc80dca51e4e62feb5cb57,1.608480,Individual,2023,2023-03-04,4,120.0,0.0,NaT,False,1437
2,00081f9797efccfbc96c5793d8de2824,14.290205,Individual,2019,2019-06-15,3,163.5,1.0,2019-06-15,False,1387
3,0008cf40d9d1da3408cff1b05571e017,10.504455,Individual,2019,2019-07-25,2,70.0,1.0,2019-07-25,False,1334
4,00095407934c7832e66c7af2e50e5541,23.179834,Individual,2019,2019-09-21,6,102.0,1.0,2019-09-21,True,1271
